In [8]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook
import engarde.decorators as ed

from linearmodels import PanelOLS

from library import test_data
@ed.verify(test_data.allyearsandtpsd)
@ed.verify(test_data.alldois)
def load(data):
    return data

from library import print_statistics

In [9]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)

In [3]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'gdid.csv'),
                  sep=",", low_memory= False)
#load(data)
data.tail()
print(data[data.doi == True].district.nunique())
data = data[data.doi == True]
data.sample()

822


,Unnamed: 0,year,campus,campname,campischarter,district,distname,distischarter,rating_academic,rating_financial,...,Unnamed: 0.1,link,p_doi,doi_year,total,_merge,doi,county,cnty_pop,treatpost
35217,35217,2016,20902044,ANGLETON J H SCHOOL,N,20902,ANGLETON ISD,N,M,Pass,...,21.0,http://angletonisd.schoolwires.net//cms/lib/TX...,0.999955,2017.0,14.0,both,True,BRAZORIA,352.0,False


In [4]:
list(data.columns)

['Unnamed: 0',
 'year',
 'campus',
 'campname',
 'campischarter',
 'district',
 'distname',
 'distischarter',
 'rating_academic',
 'rating_financial',
 'rating_academic_c',
 'type',
 'type_description',
 'cntyname',
 'students_amind_num',
 'students_asian_num',
 'students_black_num',
 'students_frpl_num',
 'students_hisp_num',
 'students_num',
 'students_paci_num',
 'students_tworaces_num',
 'students_white_num',
 'teachers_badegree_num',
 'teachers_exp_ave',
 'teachers_msdegree_num',
 'teachers_new_num',
 'teachers_nodegree_num',
 'teachers_num',
 'teachers_phddegree_num',
 'teachers_tenure_ave',
 'alg_avescore',
 'bio_avescore',
 'eng1_avescore',
 'eng2_avescore',
 'm_3rd_avescore',
 'm_4th_avescore',
 'm_5th_avescore',
 'm_6th_avescore',
 'm_7th_avescore',
 'm_8th_avescore',
 'r_3rd_avescore',
 'r_4th_avescore',
 'r_5th_avescore',
 'r_6th_avescore',
 'r_7th_avescore',
 'r_8th_avescore',
 's_8th_avescore',
 'us_avescore',
 'alg_numtakers',
 'bio_numtakers',
 'eng1_numtakers',
 'eng2_

In [5]:
data['yearpost'] = data.year - data.doi_year
data.yearpost.value_counts()

 0.0    6346
-1.0    6329
-2.0    6293
-3.0    6275
-4.0    6263
 1.0    6129
-5.0    5356
 2.0    5190
-6.0    1152
 3.0     938
-7.0     201
Name: yearpost, dtype: int64

In [6]:
#convert year to datetime
df = data.reset_index()
df['year'] = pd.to_datetime(df['year'], format='%Y')
#add column year to index
df = data.set_index(['year', 'campus'])
#swap indexes
df.index = df.index.swaplevel(0,1)
df[['district', 'doi_year','treatpost',]].tail(5)

,,district,doi_year,treatpost
campus,year,,,
253901105,2019,253901,2018.0,True
253901106,2019,253901,2018.0,True
253901107,2019,253901,2018.0,True
254902001,2019,254902,2018.0,True
254902101,2019,254902,2018.0,True


In [7]:
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + students_hisp + students_num + TimeEffects + EntityEffects', df)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

PatsyError: Error evaluating factor: NameError: name 'avescores' is not defined
    0 + avescores
        ^^^^^^^^^

# Heterogenous effects

In [25]:
### Generate variable for number of hispanic students in year before the district declares

In [26]:
data_pre = data.loc[data.yearpost == -1]
data_pre = data_pre.rename(columns = {'students_hisp': 'students_hisp_pre'})
data_pre['low_avescores_pre'] = np.where(data_pre.avescores < data_pre.avescores.quantile(.25),1, 0)
data_pre['high_avescores_pre'] = np.where(data_pre.avescores > data_pre.avescores.quantile(.75),1,0)
data_pre = data_pre[['campus','students_hisp_pre', 'low_avescores_pre', 'high_avescores_pre']]
data_hte = data.merge(data_pre, on = 'campus', how = 'left')
data_hte['majority_hisp_pre'] = np.where(data_hte.students_hisp_pre > .75, 1, 0)
#df_hte[['students_hisp', 'students_hisp_pre', 'yearpost']].sample(5)

In [27]:
data_pre = data.loc[data.year == 2016]
data_pre['rural_pre'] = np.where(data_pre.type_description == "RURAL", 1, 0)
data_pre['urban_pre'] = np.where(data_pre.type_description == "URBAN", 1, 0)
data_pre = data_pre[['campus','rural_pre', 'urban_pre']]
data_hte = data_hte.merge(data_pre, on = 'campus', how = 'left')
#df_hte[['students_hisp', 'students_hisp_pre', 'yearpost']].sample(5)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [28]:
#convert year to datetime
df_hte = data_hte.reset_index()
df_hte['year'] = pd.to_datetime(df_hte['year'], format='%Y')
#add column year to index
df_hte = data_hte.set_index(['year', 'campus'])
#swap indexes
df_hte.index = df_hte.index.swaplevel(0,1)
df_hte[['district', 'doi_year','treatpost',]].tail(20)

,,district,doi_year,treatpost
campus,year,,,
251901101,2019,251901,2017.0,True
251901104,2019,251901,2017.0,True
252901001,2019,252901,2017.0,True
252901002,2019,252901,2017.0,True
252901041,2019,252901,2017.0,True
252901101,2019,252901,2017.0,True
252901104,2019,252901,2017.0,True
252901105,2019,252901,2017.0,True
252902002,2019,252902,2017.0,True


In [31]:
df_hte['treatpost_hisp'] = df_hte.treatpost * df_hte.majority_hisp_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_hisp + students_hisp + students_num + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0107
Estimator:                   PanelOLS   R-squared (Between):              0.1206
No. Observations:               45962   R-squared (Within):              -0.0219
Date:                Wed, Oct 09 2019   R-squared (Overall):              0.1061
Time:                        18:26:53   Log-likelihood                   -4410.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      107.78
Entities:                        6121   P-value                           0.0000
Avg Obs:                       7.5089   Distribution:                 F(4,39830)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             31.479
                            

In [32]:
df_hte['treatpost_rural'] = df_hte.treatpost * df_hte.rural_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_rural + students_hisp + students_num  + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0051
Estimator:                   PanelOLS   R-squared (Between):              0.1352
No. Observations:               45349   R-squared (Within):              -0.0242
Date:                Wed, Oct 09 2019   R-squared (Overall):              0.1184
Time:                        18:26:56   Log-likelihood                   -4602.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      50.926
Entities:                        5870   P-value                           0.0000
Avg Obs:                       7.7256   Distribution:                 F(4,39468)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             12.025
                            

In [33]:
df_hte['treatpost_low'] = df_hte.treatpost * df_hte.low_avescores_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_low + students_hisp + students_num  + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0054
Estimator:                   PanelOLS   R-squared (Between):              0.1295
No. Observations:               44915   R-squared (Within):              -0.0286
Date:                Wed, Oct 09 2019   R-squared (Overall):              0.1122
Time:                        18:27:00   Log-likelihood                   -4575.1
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      52.599
Entities:                        5818   P-value                           0.0000
Avg Obs:                       7.7200   Distribution:                 F(4,39086)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             12.700
                            

In [34]:
df_hte['treatpost_high'] = df_hte.treatpost * df_hte.high_avescores_pre
mod = PanelOLS.from_formula('avescores ~ + 1 + treatpost + treatpost_high+ students_hisp + students_num  + TimeEffects + EntityEffects', df_hte)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:              avescores   R-squared:                        0.0050
Estimator:                   PanelOLS   R-squared (Between):              0.1348
No. Observations:               44915   R-squared (Within):              -0.0286
Date:                Wed, Oct 09 2019   R-squared (Overall):              0.1175
Time:                        18:27:03   Log-likelihood                   -4583.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      49.075
Entities:                        5818   P-value                           0.0000
Avg Obs:                       7.7200   Distribution:                 F(4,39086)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             11.855
                            